思路
+ 生成候选集合
+ 过滤词（使用词典过滤）
+ 使用Noisy Channer Model(语言模型)判断

In [21]:
vocab = set()  # 英文词库  # !!!!!!!!

In [23]:
# 生成所有候选集合
def generate_candidates(word):
    """
    word: 给定的输入（错误的单词）
    返回所有的（valid）候选集合
    """
    # 生成编辑距离为1的单词（插入、删除、修改）
    letters = [chr(i) for i in range(ord('a'), ord('z')+1)]
    # 1. insert
    inserts = [word[:i]+c+word[i:] for i in range(len(word)+1) for c in letters]
    # 2. delete
    deletes = [word[:i]+word[i+1:] for i in range(len(word))]
    # 3. replace
    replaces = [word[:i]+c+word[i+1:] for i in range(len(word)) for c in letters]
    
    candidates = set(inserts + deletes + replaces)
    return [word for word in candidates if word in vocab]
generate_candidates('apple')

[]

In [2]:
from nltk.corpus import reuters

# 读取语料库，训练数据
categories = reuters.categories()
corpus = reuters.sents(categories=categories)

In [7]:
# 构建语言模型：Unigram, Bigram, Trigram
# 此处使用：Bigram
term_count = {}
bigram_count = {}
for doc in corpus:
    doc = ['<s>'] + doc  # <s>开始符号，用于计算某个单词作为开头的概率
    for i in range(0, len(doc) - 1):
        term = doc[i]
        bigram = ' '.join(doc[i:i+2])
        
        if term in term_count:
            term_count[term] += 1
        else:
            term_count[term] = 1
        
        if bigram in bigram_count:
            bigram_count[bigram] += 1
        else:
            bigram_count[bigram] = 1

In [25]:
# 用户拼写错误的概率统计 - channel probability
channel_prob = {}
# 读入一个拼写错误统计文件，包含正确的单词和可能拼错的词
li_ru = {'raining': ['rainning', 'raning']}  # !!!!!!!!
for correct, mistakes in li_ru.items():
    channel_prob[correct] = {}
    for mis in mistakes:
        channel_prob[correct][mis] = 1.0/len(mistakes)
print(channel_prob)

{'raining': {'rainning': 0.5, 'raning': 0.5}}


In [27]:
import numpy as np

lines = ["How are you !", "How old are you !"]  # !!!!!!!!
for line in lines:
    words = line.split()
    for word in words:
        if word not in vocab:
            # 替换拼写错误的单词
            candidates = generate_candidates(word)
            
            probs = []
            # 对于每一个candidate,计算它的score
            # socre = P(correct)*P(mistake|correct)
            #       = log(P(correct)) + log(P(mistake|correct))
            # 返回score最大的condidate
            for candi in candidates:
                # 计算channel probability
                if candi in channel_prob and word in channel_prob[candi]:
                    prob += np.log(channel_prob[candi][word])
                else:
                    prob += np.log(0.0001)  # 此处正确的做法应该是加平滑项，使概率相加为1
                # 语言模型的概率
                idx = items[2].index(word) + 1
